# Example Analysis with OptLevAnalysis

### Import packages
Import required packages along with `AggregateData` and `FileData` from `../lib/data_processing.py`

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('../scripts/optlevstyle.mplstyle')
from data_processing import AggregateData, FileData

### Load a single file
We can use the `FileData` class directly to load and process data from a single file. Start by creating the `FileData` object, then run the `load_data()` method to populate the object attributes with the raw data.

This method calls the following:
* `read_hdf5()` to create a dictionary of the raw data
* `get_laser_power()` to get the laser power and transmitted power
* `get_diag_mat()` to extract the QPD diagonalization matrix from the config file, if the argument `diagonalize_qpd` is `True`.
* `get_xyz_from_quad()` to calculate the x, y, and z data from the QPD data, which in turn calls
    * `extract_quad()` to parse the raw QPD data into sensible arrays
* `calibrate_stage_position()` to get the cantilever positions in microns
* `calibrate_bead_response()` to apply the transfer function for both the QPD and PSPD or DC QPD. This calls one of the following:
    * `tf_array_fitted()` to get the transfer function based on fitting the poles/zeros, used for all data from 2023 and later
    * `tf_array_interpolated()` to interpolate the transfer function at the desired frequencies. Only included to reprocess Wilson data with the old transfer function format
* `get_boolean_cant_filter()` to create a filter of a specified width for selecting only the harmonics used later in the analysis
* `get_ffts_and_noise()` to get the FFTs for the cantilever and the bead, along with the noise from the sidebands
* `make_templates()` to use the bead and cantilever data to compute a signal template given a signal model.

More information on how to use each method can be found in the comments in the data processing script. After loading the data, the object should have the attributes listed after the next cell.

In [2]:
filedat = FileData('/data/new_trap/20231009/Bead0/Gravity/1/shaking_1.h5')
filedat.load_data()
# print out the class attributes
for d in dir(filedat):
    if d[:2]!='__':
        print(d)

accelerometer
bead_height
build_drive_mask
calibrate_bead_response
calibrate_stage_position
camera_status
cant_fft
cant_inds
cant_pos_calibrated
cant_raw_data
data_dict
date
diagonalize_qpd
drive_ind
drop_raw_data
error_log
extract_quad
file_name
filter_raw_data
force_cal_factors_qpd
force_cal_factors_xypd
freqs
fsamp
fund_ind
get_boolean_cant_mask
get_ffts_and_noise
get_laser_power
get_motion_likeness
get_xyz_from_quad
good_inds
is_bad
laser_power_full
load_data
make_templates
mean_cant_pos
mean_laser_power
mean_p_trans
motion_likeness
nsamp
p_trans_full
qpd_diag_mat
qpd_ffts
qpd_ffts_full
qpd_force_calibrated
qpd_sb_ffts
quad_amps
quad_null
quad_phases
quad_raw_data
read_hdf5
template_ffts
template_params
tf_array_fitted
tf_array_interpolated
times
window
window_s1
window_s2
xypd_ffts
xypd_ffts_full
xypd_force_calibrated
xypd_raw_data
xypd_sb_ffts


### Handling multiple files or datasets
The `FileData` class is intended to be called by the `AggregateData` class when loading multiple files at once.

The `AggregateData` class is designed to hold data of two types:
1. The data from each individual `FileData` object that may be needed for analysis. These data are stored in a dictionary called `agg_dict`. For each key in the dictionary, there will be a numpy array whose first dimension is the number of files loaded by the `AggregateData` object.
2. Information pertaining to subsets of the data in the `AggregateData` object rather than an individual file. These will be stored as class attributes. The arrays may have different first dimensions, since some of the datasets loaded into the `AggregateData` object may share certain parameters. For example, the attribute `diam_bead` may be an array with one entry if all files loaded used the same size bead, while the attribute `p0_bead` may have multiple entries corresponding to different subsets of the data collected with the bead at different positions relative to the attractor. The `bin_indices` attribute will allow for subsets of the data to be selected based on any of these parameters, as described in a later cell.

To initialize an `AggregateData` object, call `AggregateData()` with the following arguments:
* `data_dirs`, a list of the full paths to the directories containing the raw hdf5 files to be processed
* `file_prefixes`, a list of prefixes used to select specific files from within the directory. If you want all files from a directory, omit this argument. If you want multiple prefixes from the same directory, repeat the directory path in the `data_dirs` argument and add the corresponding file prefixes.
* `descrips`, a list of descriptions intended to be useful to the user in identifying different dataset
* `num_to_load` a list of the number of files to load from each directory, or an integer to load the same number from all directories. If this argument is omitted, all files from each directory will be included.

In the example below, an `AggregateData` object for 100 files from the Wilson data and a dataset from June 14, 2023 is initialized.


In [3]:
aggdat = AggregateData(['/data/new_trap/20231009/Bead0/Gravity/1/','/data/new_trap/20200320/Bead1/Shaking/Shaking378/'],\
                       file_prefixes=['shaking_','Shaking1'],descrips=['Oct 2023','Wilson'],num_to_load=[100,100])

### Loading the data
To load and preprocess the data, call the `load_file_data()` method with the following arguments:
* `num_cores`, the number of cores to use for processing the files in parallel
* `diagonalize_qpd`, whether to look in the config file for a diagonalization matrix to use for the calculation of $x$ and $y$ from the QPD.
* `load_templates`, whether to load the signal templates for each file.
* `harms`, a list of harmonics to be used in the analysis.
* `no_tf` and `no_config`, whether to load the file data without looking for a config file or transfer function data. If `True`, default parameters will be used.
* `lightweight`, a boolean parameter which determines whether or not to keep the raw data as a class attribute. If set to False, only the reduced data will be accessible within the `AggregateData` object.

This method also takes care of a few additional organizational steps:
* Saves the full list of files imported as a class attribute
* Loads the bead positions `p0_bead` and diameters `diam_bead` from a config file, `config.yaml`, located in the same directory as the hdf5 files. Removes any duplicate entries for easier indexing (described in more detail further down).
* Identifies any bad files, saves a list of the filenames, and purges them from the `AggregateData` object

At this point a list of the `FileData` objects has been loaded, so all the `FileData` attributes can be accessed, albeit in a clunky way. To make accessing these parameters for many datasets at a time easier, a private methoc called `__build_dict()` will extract all the data from the `FileData` objects and put it into the `agg_dict` attribute of the `AggregateData` class.

In [4]:
aggdat.load_file_data(num_cores=20,lightweight=False,downsample=False)
aggdat.agg_dict.keys()

Loading data from 200 files...


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:06<00:00, 31.73it/s]


Successfully loaded 196 files.
Building dictionary of file data...
Done building dictionary.


dict_keys(['freqs', 'fsamp', 'nsamp', 'window_s1', 'window_s2', 'fund_ind', 'good_inds', 'times', 'timestamp', 'accelerometer', 'bead_height', 'mean_laser_power', 'laser_power_full', 'mean_p_trans', 'p_trans_full', 'cant_raw_data', 'quad_raw_data', 'quad_null', 'mean_cant_pos', 'qpd_ffts', 'qpd_ffts_full', 'qpd_sb_ffts', 'xypd_ffts', 'xypd_ffts_full', 'xypd_sb_ffts', 'template_ffts', 'template_params', 'cant_fft', 'quad_amps', 'quad_phases', 'mot_likes', 'axis_angles', 'camera_status'])

### Binning by auxiliary data
Ultimately we may want to load many datasets but look only at a subset of that data that meets some criteria for a particular analysis. Rather than creating multiple sparse, nested arrays, data for all files loaded is kept in a single 1-D array, and a secondary array is used to track the indices. For each file loaded, there is a corresponding row in the array of indices that includes the following information:
* `diam_bead` index: The index that can be used to select the correct bead diameter value from the array containing all values for the `AggregateData` object.
* `p0_bead` index: Same thing, but for the bead position.
* `descrips` index: Same thing, but for the descriptions passed in for each directory when the constructor was called.
* `cant_bins_x` index: An index that determines which bin of mean cantilever x positions the file falls into.
* `cant_bins_z` index: Same thing, but for the mean cantilever z positions.
* `seis_thresh` index: 0 if the file passes the seismometer threshold cut, 1 if it doesn't.
* `bias` index: Not yet implemented, but will eventualy index the attractor/shield bias.
* `qpd_asds` and `xypd_asds` indices (the same for both): Index of the RMS spectral densities for that subset of the data.
* `is_bad` index: 1 if the file is bad, 0 otherwise. This is used for purging, so in principle after the data is loaded all values should be 0.

The method `bin_by_aux_data()` takes the following arguments:
* `cant_bin_widths`: a list containing the bin widths for the cantilever x and z positions. The data will be histogrammed into bins of the specified widths, then empty bins will be removed and the indices adjusted accordingly.
* `seis_thresh`: the threshold value used in the seismometer data cut.

Once this method has been called, the `bin_indices` attribute should be populated with indices pertaining to the auxiliary data, and can be used for indexing arbitrary slices.

In [5]:
aggdat.bin_by_aux_data(cant_bin_widths=[1.,1.],accel_thresh=0.01)
print(aggdat.bin_indices[:5])
print(aggdat.bin_indices[-5:])

Binning data by mean cantilever position and accelerometer data...
Done binning data.
[[0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]
[[1 0 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 0 0 0 0 0]]


### Estimating the RMS spectral densities for large datasets
With the full FFT data for each file loaded, the `AggregateData` objects can become quite large. Since the individual FFTs for each dataset may not be needed, the method `estimate_spectral_densities()` can be used to compute the RMS average of the amplitude spectral densities for each combination of run conditions. This will add class attributes `qpd_asds` and `xypd_asds`, arrays whose first dimension represents the subset of the data to which the spectra correspond, second dimension represents the axis, and third dimension represents the frequency. Corresponding indices will be added to the `bin_indices` attribute so that it can be easily determined which file contributes to which RMS spectral density.

Once this has been run, the method `drop_full_ffts()` can be used to delete the full FFT data for each file and release memory.

In [6]:
aggdat.estimate_spectral_densities()
print('AggregateData spectral density shapes:')
print(aggdat.qpd_asds.shape)
print(aggdat.xypd_asds.shape)
print('Dictionary column shapes before dropping FFT data:')
print(aggdat.agg_dict['qpd_ffts_full'].shape)
print(aggdat.agg_dict['xypd_ffts_full'].shape)
aggdat.drop_full_ffts()
print('After dropping FFT data:')
print(aggdat.agg_dict['qpd_ffts_full'].shape)
print(aggdat.agg_dict['xypd_ffts_full'].shape)

Estimating RMS amplitude spectral density for each set of run conditions...
Amplitude spectral densities estimated for 3 sets of run conditions.
AggregateData spectral density shapes:
(3, 3, 5001)
(3, 2, 5001)
Dictionary column shapes before dropping FFT data:
(196, 4, 5001)
(196, 3, 5001)
After dropping FFT data:
(196, 4, 1)
(196, 3, 1)


### Indexing data with auxiliary parameters

The config data and auxiliary parameters are not saved in `agg_dict` to avoid saving the same values many times over and wasting memory. However, it may be convenient to get arrays of these parameters for use in masking/indexing the data in `agg_dict`. The method `get_parameter_arrays()` will return a tuple of arrays of all the config data and auxiliary parameters. The arrays will have the same first dimension as the data in `agg_dict` so that they can be used in indexing. The method will also print out the order in which the arrays are returned. The cell below shows an example of how the `mean_laser_power` data can be masked to select only those files with a 7.6 micron diameter bead from the October 2023 data.

In [7]:
# we only care about the first array returned in this case
diameters,_,descrips,*_ = aggdat.get_parameter_arrays()

# now use the array of diameters to index within agg_dict
laser_power = aggdat.agg_dict['mean_laser_power'][(diameters==7.6) & (descrips=='Oct 2023')]
print()
print(laser_power)

Returning a tuple of the following arrays:
(diam_bead, mass_bead, p0_bead, descrips, qpd_diag_mats, cant_bins_x, cant_bins_z)

[]


/tmp/ipykernel_25335/545293291.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  laser_power = aggdat.agg_dict['mean_laser_power'][(diameters==7.6) & (descrips=='Oct 2023')]


### Indexing using the built-in method
The `get_slice_indices()` method is designed as a user-friendly wrapper for this type of slicing. This method takes the following arguments:
* `diam_bead`: a float or a list of floats containing the desired bead diameters in microns.
* `descrips`: a string or list of strings containing the descriptions of the datasets chosen when the constructor was called.
* `cant_x`: a range of mean cantilever x positions in microns to include, of the form `[lower,upper]`.
* `cant_z`: same as above for the mean cantilever z positions.
* `seis_veto`: a boolean that says whether or not data not passing the seismometer cut should be included.

This method returns an array of the indices of files that pass the cuts. As an example, in the cell below we create separate index arrays for the Wilson data and the October 2023 data and require that all files from both pass the siesmometer cut. This array of indices is then used to compare the QPD x and y spectra.

In [8]:
# get arrays of indices
indices_wilson = aggdat.get_slice_indices(descrip='Wilson')
indices_2023 = aggdat.get_slice_indices(descrip='Oct 2023')
print('Number of files from Wilson data: ',len(indices_wilson))
print('Number of files from Oct 2023 data: ',len(indices_2023))

Number of files from Wilson data:  100
Number of files from Oct 2023 data:  96


### Miscellaneous file handling methods
`AggregateData` objects can be saved and loaded using the `save_to_hdf5()` and `load_from_hdf5()` methods respectively. The only argument to both is the path to where the object should be/was saved. Since the loading method relies on the object having been saved in a particular format, it may not work for objects that were saved with a different version of the code. However, as objects are saved in HDF5 format, the file can be loaded using `h5py` and the datasets extracted based on the (hopefully self-explanatory) descriptors.

Any number of `AggregateData` objects can be merged into one after the fact using the `merge_objects()` method with a list of objects as the argument, so that if more datasets need to be added the existing data does not need to be reprocessed. This should be called by constructing a new `AggregateData` object first and then calling the method to fill it with the existing objects.